## Time series forecasting

Time series forecasting involves using historical time series data to predict future values of the series. Machine learning models offer several advantages for time series forecasting:
* can hadle complex temporal patterns
* can integrate multiple types/sources of data
* can adapt to new trends over time
* are easy to implement

In this module we will extend the river discharge problem in the time dimension and try to predict the discharge in the next time step using the information available at the present time.

**1. Load libraries and data**

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.metrics import mean_squared_error

import random

In [ ]:
# Read data 
file_url = 'https://github.com/DHI/Intro_ML_course/raw/main/module_5/danube_discharge.csv'
df = pd.read_csv(file_url, parse_dates=True, index_col=0)

**2. Feature engineering**

First of all, we need to transform the data in a format that provides the most information for predicting the river discharge. Given the temporal dynamics we expect that the aggregate temperature and precipitation values can be more informative than the daily ones.

In [ ]:
df['precip_weekly_sum'] = df['precipitation'].rolling(7).sum()
df['precip_monthly_sum'] = df['precipitation'].rolling(30).sum()
df['precip_quartelry_sum'] = df['precipitation'].rolling(120).mean()
df['temperature_weekly_mean'] = df['temperature'].rolling(7).mean()
df['temperature_monthly_mean'] = df['temperature'].rolling(30).mean()
df['temperature_quarterly_mean'] = df['temperature'].rolling(120).mean()
df['month_sin'] = np.sin(2*np.pi*(df.index.month)/12)
df['month_cos'] = np.cos(2*np.pi*(df.index.month)/12)
df['week_sin'] = np.sin(2*np.pi*(df.index.day_of_year/7)/52)
df['week_cos'] = np.cos(2*np.pi*(df.index.day_of_year/7)/52)

We can then check how these features are correlated with the discharge to select the best predictors.

In [ ]:
# Compute the pairwise cross-correlation matrix of all columns
cross_correlation_matrix = df.corr()

cross_correlation_matrix.discharge.sort_values(ascending=False)

In this example we will use as input features all the variables with absolute correlation higher than 0.1, except for the discharge itself, and set as target the discharge at the following time step.

In [ ]:
df_ANN = df[['discharge', 'precip_monthly_sum', 'precip_quartelry_sum',
             'temperature_monthly_mean', 'temperature_quarterly_mean', 
             'month_sin', 'month_cos', 'week_sin', 'week_cos']].copy()
df_ANN.columns = ['y_t', 'x1_t', 'x2_t', 'x3_t', 'x4_t', 'x5_t', 'x6_t', 'x7_t', 'x8_t']
df_ANN['y_t+1'] = df_ANN['y_t'].shift(-1)

# Reorder columns
df_ANN = df_ANN[['x1_t', 'x2_t', 'x3_t', 'x4_t', 'x5_t', 'x6_t', 'x7_t', 'x8_t', 'y_t+1']] # Notice we have dropped y_t (discharge)

# Remove initial and final rows with missing values
df_ANN = df_ANN[~df_ANN.isnull().any(axis=1)]

df_ANN.head(6)

In [ ]:
df_ANN.tail(6)

Next, we will separate input and target values, normalize the data and split it sequentially in training, validation and test sets.

In [ ]:
X = df_ANN.iloc[:, :-1].values
y = df_ANN.iloc[:, -1].values

data_length = len(X)
train_end = int(data_length * 0.7) # 70% of data is used for training
validation_end = int(data_length * 0.85) # 15% of data is used for validation and test

train_end, validation_end, data_length

In [ ]:
X_train = X[:train_end]
X_val = X[train_end:validation_end]
X_test = X[validation_end:]

X_train.shape, X_test.shape, X_val.shape

y_train = y[:train_end]
y_val = y[train_end:validation_end]
y_test = y[validation_end:]

y_train.shape, y_test.shape, y_val.shape

In [ ]:
# Normalize the data
scaler_X = MinMaxScaler()

# Fit the scaler on the training data and transform both training and testing data
X_train_scaled = scaler_X.fit_transform(X_train)
X_val_scaled = scaler_X.transform(X_val)
X_test_scaled = scaler_X.transform(X_test)

scaler_y = MinMaxScaler()

y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))
y_val_scaled = scaler_y.transform(y_val.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))

X_train_scaled.shape, X_val_scaled.shape, X_test_scaled.shape

Finally, we can compile and train the MLP (ANN) with our choice of architecture and hyperparameters.

In [ ]:
random.seed(42)

# Building the ANN model
model = Sequential([
    Dense(units=16, activation='relu', input_dim=8),
    Dropout(0.1), # Read about dropout layers here: https://keras.io/api/layers/regularization_layers/dropout/
    Dense(units=16, activation='relu'),
    Dropout(0.1),
    Dense(units=1, activation='linear')
])

# Model summary
model.summary()

# Compiling the model
model.compile(optimizer=Adam(learning_rate=0.00001), loss='mean_squared_error')

# Training the model
history = model.fit(X_train_scaled, y_train_scaled, 
                    batch_size=2, epochs=50, 
                    validation_data=(X_val_scaled, y_val_scaled), verbose=1)

To evaluate the performance of the model we will plot the learning curve, compute the RMSE on the test predictions and visualize the predictions against the actual values.

In [ ]:
# Plot learning curves
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.ylabel('MSE')
plt.xlabel('Epoch')
plt.legend()
plt.title('Learning curves')
plt.show()

# Make predictions for the test dataset
y_pred_scaled = model.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
predictions = y_pred.reshape(-1)

# Print MSE score of test predictions
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print('RMSE (test): ', rmse)

# Plot the predictions against the actual values
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_ANN.index[validation_end:], y=y_test, name='Observed', line=dict(color='black')))
fig.add_trace(go.Scatter(x=df_ANN.index[validation_end:], y=predictions, name='Predicted (test)', line=dict(color='red')))
fig.update_yaxes(title_text="Discharge (m3/s)")
fig.update_xaxes(title_text="Day")
fig.update_layout(margin=dict(l=20, r=20, t=20, b=20), legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig.show()

In [ ]:
# Compare on the same plot the prediction for the train, validation and test datasets
y_train_pred_scaled = model.predict(X_train_scaled)
y_train_pred = scaler_y.inverse_transform(y_train_pred_scaled)
predictions_train = y_train_pred.reshape(-1)

y_val_pred_scaled = model.predict(X_val_scaled)
y_val_pred = scaler_y.inverse_transform(y_val_pred_scaled)
predictions_val = y_val_pred.reshape(-1)

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_ANN.index, y=df_ANN['y_t+1'], name='Observed', line=dict(color='black')))
fig.add_trace(go.Scatter(x=df_ANN.index[:train_end], y=predictions_train, name='Predictions (train)', line=dict(color='green')))
fig.add_trace(go.Scatter(x=df_ANN.index[train_end:validation_end], y=predictions_val, name='Predictions (validation)', line=dict(color='violet')))
fig.add_trace(go.Scatter(x=df_ANN.index[validation_end:], y=predictions, name='Predictions (test)', line=dict(color='red')))
fig.update_yaxes(title_text="Discharge (m3/s)")
fig.update_xaxes(title_text="Day")
fig.update_layout(margin=dict(l=20, r=20, t=20, b=20), legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig.show()